In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import mplleaflet

from geopy.distance import geodesic

import os

import math

from k_medoids import KMedoidsWeighted

%matplotlib inline

In [2]:
data_dir = '../data/'
mamografo_dataset_name = 'RF- Mamógrafos.csv'
use_cols = ['COD_MUN','2018/Dez'] # obtendo os dados mais recentes
municipios_dataset_name = 'dados_mun.csv'
tqdm.pandas()

In [3]:
def dist_to_near_mamografo(row):
    '''
    Retorna a menor distancia a um mamografo, dada a latitude e longitude de uma cidade
    '''
    distancias = cities_mamografo[cities_mamografo.UF == row.UF].apply(lambda x: geodesic((x['LATITUDE'], x['LONGITUDE']), (row.LATITUDE, row.LONGITUDE)).km , axis=1)
    index_min = distancias.idxmin()
    near_city = cities_mamografo.iloc[index_min]
    distancia = geodesic((near_city.LATITUDE, near_city.LONGITUDE), (row.LATITUDE, row.LONGITUDE)).km
    return near_city.COD_MUN, distancia
def population_around_mamografo(codigo_mun):
    '''
    Retorna a soma das populacoes das cidades ao redor de um mamografo
    '''
    cidades_proximas = municipios[municipios.COD_PROX == codigo_mun]
    return cidades_proximas.POPULACAO.astype(int).sum()

In [4]:
# Lendo os dataframes:

mamografo = pd.read_csv(os.path.join(data_dir, mamografo_dataset_name), encoding='latin', index_col=False, 
                     error_bad_lines=False, low_memory=False, usecols=use_cols)

municipios = pd.read_csv(os.path.join(data_dir, municipios_dataset_name), encoding='latin', index_col=0, 
                     error_bad_lines=False, low_memory=False)

municipios['COD_MUN'] = municipios['COD_MUN'].apply(lambda x: (int) ((x - x%10)/10))
cities_mamografo = pd.merge(mamografo[mamografo['2018/Dez'] > 0], municipios, on='COD_MUN', how='left')
municipios = municipios.assign(DISTANCIAS = municipios.progress_apply(lambda row: dist_to_near_mamografo(row)[1], axis=1), COD_PROX = municipios.progress_apply(lambda row: dist_to_near_mamografo(row)[0], axis=1))
estados = municipios.UF.unique()
distancia_max = 70
populacao_max = 100000

In [21]:
UF_analisado = 'AP'
cidades_no_estado = municipios[municipios.UF == UF_analisado]
proximos, distantes = cidades_no_estado[cidades_no_estado.DISTANCIAS < distancia_max], cidades_no_estado[cidades_no_estado.DISTANCIAS > distancia_max]
mamografo_no_estado = cities_mamografo[cities_mamografo.UF == UF_analisado]
mamografo_no_estado = mamografo_no_estado.assign(POP_ATENDIDA = mamografo_no_estado.COD_MUN.progress_apply(lambda cod: population_around_mamografo(cod)))
pop_ok, pop_n_ok = mamografo_no_estado[mamografo_no_estado.POP_ATENDIDA < populacao_max], mamografo_no_estado[mamografo_no_estado.POP_ATENDIDA > populacao_max]

In [22]:
plt.scatter(distantes.LONGITUDE, distantes.LATITUDE, color='r' , s = 70) # plotar cidades com mamografos distantes
plt.scatter(proximos.LONGITUDE, proximos.LATITUDE, color='b',s = 70) # plotar cidades com mamografos proximos
plt.scatter(pop_ok.LONGITUDE, pop_ok.LATITUDE, marker='^', color='yellowgreen', s = 400) # plotar cidades em que o numero de mamografos sao adequados
plt.scatter(pop_n_ok.LONGITUDE, pop_n_ok.LATITUDE, marker='^', color='darkolivegreen', s = 400) #plotar cidades em que o numero de mamografos nao sao adequados
mplleaflet.show()

In [7]:
total_cidades, total_populacao, total_mamografos, ideal_mamografos,suficiente,cidades_proximas, cidades_distantes, centros_suficiente, centros_insuficiente = [],[],[],[],[],[],[],[],[]
for estado in estados:
    UF_analisado = estado
    cidades_no_estado = municipios[municipios.UF == estado]
    mamografo_no_estado = cities_mamografo[cities_mamografo.UF == estado]
    mamografo_no_estado = mamografo_no_estado.assign(POP_ATENDIDA = mamografo_no_estado.COD_MUN.apply(lambda cod: population_around_mamografo(cod)))
    
    distantes, proximas = cidades_no_estado[cidades_no_estado.DISTANCIAS > distancia_max], cidades_no_estado[cidades_no_estado.DISTANCIAS < distancia_max]
    suficientes, insuficientes = mamografo_no_estado[mamografo_no_estado.POP_ATENDIDA < populacao_max], mamografo_no_estado[mamografo_no_estado.POP_ATENDIDA > populacao_max]
    total_populacao_estado = cidades_no_estado.POPULACAO.astype(int).sum()
    total_mamografo_estado = mamografo_no_estado['2018/Dez'].astype(int).sum()
    ideal_mamografo_estado = math.ceil(total_populacao_estado/populacao_max)
    
    total_cidades.append(cidades_no_estado.COD_MUN.count())
    total_populacao.append(total_populacao_estado)
    total_mamografos.append(total_mamografo_estado)
    ideal_mamografos.append(ideal_mamografo_estado)
    suficiente.append(total_mamografo_estado > ideal_mamografo_estado)
    cidades_proximas.append(proximas.COD_MUN.count())
    cidades_distantes.append(distantes.COD_MUN.count())
    centros_suficiente.append(suficientes.COD_MUN.count())
    centros_insuficiente.append(insuficientes.COD_MUN.count())
    
panorama_distribuicao_mamografos_uf = pd.DataFrame({
    'UF': estados,
    'TOTAL_CIDADES': total_cidades,
    'TOTAL_POPULACAO': total_populacao,
    'TOTAL_MAMOGRAGOS': total_mamografos,
    'IDEAL_MAMOGRAFOS': ideal_mamografos,
    'SUFICIENTE': suficiente,
    'CIDADES_PROXIMAS': cidades_proximas,
    'CIDADES_DISTANTES': cidades_distantes,
    'CENTROS_SUFICIENTES': centros_suficiente,
    'CENTROS_INSUFICIENTES': centros_insuficiente
},columns= ['UF', 'TOTAL_MAMOGRAGOS', 'TOTAL_POPULACAO', 'IDEAL_MAMOGRAFOS','SUFICIENTE','TOTAL_CIDADES', 'CIDADES_DISTANTES' , 'CIDADES_PROXIMAS', 'CENTROS_SUFICIENTES', 'CENTROS_INSUFICIENTES'])
panorama_distribuicao_mamografos_uf

,UF,TOTAL_MAMOGRAGOS,TOTAL_POPULACAO,IDEAL_MAMOGRAFOS,SUFICIENTE,TOTAL_CIDADES,CIDADES_DISTANTES,CIDADES_PROXIMAS,CENTROS_SUFICIENTES,CENTROS_INSUFICIENTES
0,RO,14,1768204,18,False,52,29,23,0,5
1,AC,3,803513,9,False,22,14,8,0,2
2,AM,55,3938336,40,True,62,15,47,29,7
3,RR,4,505665,6,False,15,12,3,0,1
4,PA,50,8206923,83,False,144,44,100,1,22
5,AP,2,766679,8,False,16,13,3,0,1
6,TO,18,1515126,16,True,139,80,59,1,6
7,MA,30,6904241,70,False,217,92,125,0,15
8,PI,40,3204028,33,True,224,81,143,3,10
9,CE,69,8904459,90,False,184,14,170,1,24


In [8]:
# panorama_distribuicao_mamografos_uf.CIDADES_DISTANTES / panorama_distribuicao_mamografos_uf.TOTAL_CIDADES
# aplicar o k-medoids para definir novos pontos


0     0.557692
1     0.636364
2     0.241935
3     0.800000
4     0.305556
5     0.812500
6     0.575540
7     0.423963
8     0.361607
9     0.076087
10    0.041916
11    0.004484
12    0.086486
13    0.000000
14    0.013333
15    0.112710
16    0.048066
17    0.000000
18    0.000000
19    0.003101
20    0.022556
21    0.013559
22    0.012072
23    0.341772
24    0.673759
25    0.154472
26    0.000000
dtype: float64

In [ ]:
# nova distribuicao de recursos nos estados